In [ ]:
# booking.py
from tkinter import *
import sqlite3
import tkinter.messagebox
from datetime import datetime, timedelta

# scheduler
try:
    from apscheduler.schedulers.background import BackgroundScheduler
    scheduler = BackgroundScheduler()
    scheduler.start()
    APS_AVAILABLE = True
except Exception:
    scheduler = None
    APS_AVAILABLE = False
    print("APScheduler not available. Reminders will be skipped. Run `pip install apscheduler` to enable.")

# DB connection
DB_PATH = 'database.db'
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

# ensure table exists (safe, doesn't alter existing schema)
c.execute("""
CREATE TABLE IF NOT EXISTS appointments(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER,
    gender TEXT,
    location TEXT,
    scheduled_time TEXT,
    phone TEXT
)
""")
conn.commit()

# helper to schedule a GUI popup reminder in a thread-safe way
def schedule_gui_popup(master, remind_at, name, date):
    """
    Schedules a popup reminder using the background scheduler.
    The scheduled job will call master.after(0, show_popup) so the messagebox runs on the main thread.
    """
    if not APS_AVAILABLE:
        print("APScheduler not installed — skipping scheduling.")
        return
    if remind_at <= datetime.now():
        print("Reminder time is in the past. Skipping.")
        return

    def enqueue_popup():
        master.after(0, lambda: tkinter.messagebox.showinfo("Booking Reminder", f"Reminder: LPG booking for {name} on {date}!"))

    # schedule the job (runs in background thread, but only enqueues popup on main thread)
    scheduler.add_job(enqueue_popup, 'date', run_date=remind_at)
    print("Reminder scheduled for:", remind_at)


class Application:
    def __init__(self, master):
        self.master = master

        # creating the frames in the master
        self.left = Frame(master, width=800, height=720, bg='grey')
        self.left.pack(side=LEFT)

        self.right = Frame(master, width=600, height=720, bg='black')
        self.right.pack(side=RIGHT)

        # labels for the window
        self.heading = Label(self.left, text="LPG Booking System", font=('georgia 40 bold'), fg='black', bg='grey')
        self.heading.place(x=0, y=0)

        # customers name
        self.name = Label(self.left, text="Customer's Name", font=('georgia 18 bold'), fg='black', bg='grey')
        self.name.place(x=0, y=100)

        # age
        self.age = Label(self.left, text="Age", font=('georgia 18 bold'), fg='black', bg='grey')
        self.age.place(x=0, y=140)

        # gender
        self.gender = Label(self.left, text="Gender", font=('georgia 18 bold'), fg='black', bg='grey')
        self.gender.place(x=0, y=180)

        # location
        self.location = Label(self.left, text="Address", font=('georgia 18 bold'), fg='black', bg='grey')
        self.location.place(x=0, y=220)

        # appointment date
        self.time = Label(self.left, text="Book date", font=('georgia 18 bold'), fg='black', bg='grey')
        self.time.place(x=0, y=260)

        # phone
        self.phone = Label(self.left, text="Phone Number", font=('georgia 18 bold'), fg='black', bg='grey')
        self.phone.place(x=0, y=300)

        # Entries for all labels
        self.name_ent = Entry(self.left, width=30)
        self.name_ent.place(x=250, y=100)

        self.age_ent = Entry(self.left, width=30)
        self.age_ent.place(x=250, y=140)

        self.gender_ent = Entry(self.left, width=30)
        self.gender_ent.place(x=250, y=180)

        self.location_ent = Entry(self.left, width=30)
        self.location_ent.place(x=250, y=220)

        self.time_ent = Entry(self.left, width=30)
        self.time_ent.place(x=250, y=260)

        self.phone_ent = Entry(self.left, width=30)
        self.phone_ent.place(x=250, y=300)

        # button to perform a command
        self.submit = Button(self.left, text="Add Booking", width=20, height=2, bg='white', command=self.add_appointment)
        self.submit.place(x=300, y=340)

        # getting the number of appointments fixed to view in the log
        sql2 = "SELECT ID FROM appointments"
        try:
            self.result = c.execute(sql2).fetchall()
            ids = [row[0] for row in self.result] if self.result else []
            if ids:
                self.final_id = max(ids)
            else:
                self.final_id = 0
        except Exception:
            self.final_id = 0

        # displaying the logs in our right frame
        self.logs = Label(self.right, text="Booking logs", font=('georgia 28 bold'), fg='black', bg='grey')
        self.logs.place(x=70, y=10)

        self.box = Text(self.right, width=60, height=30)   # smaller width to fit nicer
        self.box.place(x=20, y=60)
        self.box.insert(END, "Total Bookings till now : " + str(self.final_id) + " \n\n")

        # show recent bookings in the text box (simple)
        try:
            c.execute("SELECT id, name, scheduled_time FROM appointments ORDER BY id DESC LIMIT 20")
            rows = c.fetchall()
            for r in rows:
                self.box.insert(END, f"{r[0]} - {r[1]} ---> {r[2]}\n")
        except Exception:
            pass

    # function called when the submit button is clicked
    def add_appointment(self):
        # getting the user inputs
        self.val1 = self.name_ent.get().strip()
        self.val2 = self.age_ent.get().strip()
        self.val3 = self.gender_ent.get().strip()
        self.val4 = self.location_ent.get().strip()
        self.val5 = self.time_ent.get().strip()
        self.val6 = self.phone_ent.get().strip()

        # checking if the user input is empty
        if self.val1 == '' or self.val2 == '' or self.val3 == '' or self.val4 == '' or self.val5 == '':
            tkinter.messagebox.showinfo("Warning", "Please Fill Up All Boxes")
            return

        # basic validation: age numeric
        if self.val2 and not self.val2.isdigit():
            tkinter.messagebox.showerror("Invalid input", "Age must be a number.")
            return

        # insert to the database
        try:
            sql = "INSERT INTO appointments (name, age, gender, location, scheduled_time, phone) VALUES(?, ?, ?, ?, ?, ?)"
            c.execute(sql, (self.val1, int(self.val2) if self.val2 else None, self.val3 or None, self.val4 or None, self.val5, self.val6 or None))
            conn.commit()
            tkinter.messagebox.showinfo("Success", "Booking for " + str(self.val1) + " has been created")
            self.box.insert(END, 'Booking fixed for ' + str(self.val1) + ' ---> ' + str(self.val5) + '\n')
        except Exception as e:
            tkinter.messagebox.showerror("DB Error", str(e))
            return

        # schedule GUI popup reminder (1 day before booking date)
        try:
            # For quick testing you can change this to datetime.now() + timedelta(minutes=1)
            booking_date = datetime.strptime(self.val5, "%Y-%m-%d")
            remind_time = booking_date - timedelta(days=1)

            # schedule the popup using the helper (master is self.master)
            schedule_gui_popup(self.master, remind_time, self.val1, self.val5)
        except ValueError:
            print("Could not schedule reminder — invalid date format. Use YYYY-MM-DD.")
        except Exception as e:
            print("Reminder scheduling failed:", e)

        # clear fields after adding
        self.name_ent.delete(0, END)
        self.age_ent.delete(0, END)
        self.gender_ent.delete(0, END)
        self.location_ent.delete(0, END)
        self.time_ent.delete(0, END)
        self.phone_ent.delete(0, END)


# creating the object
if __name__ == "__main__":
    root = Tk()
    b = Application(root)
    root.geometry("1366x768")
    root.resizable(False, False)
    root.mainloop()

# note: connection left open while app runs; it will close on program exit
